# Module 07 — Datasheet Reading Methodology

## A General Framework for Reading Any Semiconductor Datasheet

You have spent six modules learning about diodes, BJTs, MOSFETs, op-amps, logic ICs,
and more. Every one of those topics required you to look at **datasheets** — the
manufacturer's definitive document describing what a component can and cannot do.

This notebook distills everything you have learned into a **repeatable, 5-step method**
you can apply to *any* semiconductor datasheet you encounter in the future.

---
## 1. Why Datasheets Matter

The datasheet is the **contract** between you and the manufacturer.

- It tells you what the device **is guaranteed** to do (min/max specs).
- It tells you what the device **typically** does (nominal behavior).
- It tells you where the device **will break** (absolute maximum ratings).
- It provides **application guidance** (reference circuits, layout tips, thermal design).

If your circuit violates any datasheet limit, you have **no guarantee** the device will
work — or survive. The manufacturer owes you nothing outside the datasheet.

> **Rule of thumb:** If it is not in the datasheet, do not assume it.

---
## 2. The 5-Step Datasheet Reading Method

Every datasheet — whether it is a $0.02 diode or a $200 FPGA — can be attacked
with the same five steps.

### Step 1: IDENTIFY — What Is This Device?

| Question | Where to look |
|---|---|
| What **type** of device? (diode, BJT, MOSFET, IC, etc.) | Title, description, features list |
| What **package**? (SOT-23, TO-220, DIP-8, QFN, etc.) | Package section, ordering info |
| What is the **pinout**? | Pin diagram, pin function table |
| What is the **basic function**? | Features, functional block diagram |

### Step 2: LIMITS — Absolute Maximum Ratings

These are the **"do not cross" lines**. Exceeding any single one, even briefly, can
destroy the device.

Common absolute maximum parameters:
- **Voltage** — maximum voltage across any pin pair
- **Current** — maximum current through any pin
- **Temperature** — storage and operating junction temperature
- **Power dissipation** — maximum total power the package can handle
- **ESD rating** — electrostatic discharge tolerance

### Step 3: TYPICAL OPERATION — Recommended Operating Conditions

These are the conditions the manufacturer **designed and tested** the device for.
The electrical characteristics table gives you guaranteed performance **only at these
conditions**.

- Supply voltage range
- Operating temperature range
- Input/output voltage and current ranges

### Step 4: CURVES — Characteristic Graphs

Graphs show how parameters **change** with temperature, voltage, current, or frequency.
They tell the story the tables cannot.

- **I-V curves** — diode forward voltage, transistor output characteristics
- **Gain vs. frequency** — op-amp open-loop gain, transistor f_T
- **Derating curves** — how power or current drops with temperature
- **Safe Operating Area (SOA)** — the voltage/current envelope the device can handle
- **Thermal impedance** — transient thermal behavior for pulsed loads

### Step 5: APPLICATION — Reference Circuits and Design Notes

Many datasheets include:
- **Reference/typical circuits** — a known-good starting point
- **Component selection guides** — how to choose external Rs, Cs, Ls
- **Layout recommendations** — ground planes, decoupling, trace width
- **Thermal design** — heatsink requirements, copper area for heat spreading

---
## 3. Common Datasheet Parameters by Device Type

Every device type has its own "big five" parameters. The table below summarizes
what to look for first when you open a datasheet.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Summary comparison table of key parameters by device type
device_types = {
    "Diode": {
        "Key Parameters": ["V_F (forward voltage)", "I_F (forward current)",
                           "V_R (reverse voltage)", "I_R (reverse leakage)",
                           "t_rr (reverse recovery time)", "C_j (junction capacitance)"],
        "First Question": "What voltage drop and how much current?",
    },
    "BJT": {
        "Key Parameters": ["h_FE (current gain)", "V_CE(sat) (saturation voltage)",
                           "V_BE (base-emitter voltage)", "f_T (transition frequency)",
                           "SOA (safe operating area)"],
        "First Question": "What gain, and how much current can it switch?",
    },
    "MOSFET": {
        "Key Parameters": ["V_GS(th) (threshold voltage)", "R_DS(on) (on-resistance)",
                           "Q_g (gate charge)", "I_D (drain current)",
                           "SOA (safe operating area)"],
        "First Question": "What voltage to turn it on, and how low is the on-resistance?",
    },
    "Op-Amp": {
        "Key Parameters": ["V_OS (offset voltage)", "A_OL (open-loop gain)",
                           "GBW (gain-bandwidth product)", "CMRR (common-mode rejection)",
                           "Slew rate (V/us)"],
        "First Question": "How fast and how precise?",
    },
    "Logic IC": {
        "Key Parameters": ["V_OH / V_OL (output levels)", "V_IH / V_IL (input thresholds)",
                           "t_PD (propagation delay)", "I_CC (supply current)",
                           "f_MAX (max clock frequency)"],
        "First Question": "What logic levels, and how fast?",
    },
}

# Print formatted table
print("=" * 90)
print(f"{'Device Type':<12} {'First Question':<55} {'# Key Params'}")
print("=" * 90)
for dev, info in device_types.items():
    print(f"{dev:<12} {info['First Question']:<55} {len(info['Key Parameters'])}")
    for p in info["Key Parameters"]:
        print(f"{'':>14}- {p}")
    print("-" * 90)

---
## 4. Test Conditions Are Everything

A datasheet parameter is **meaningless** without its test conditions.

Example: a MOSFET datasheet says R_DS(on) = 0.044 ohm.

But look at the fine print:
- **V_GS = 10V, I_D = 25A, T_J = 25 degC** — that is the test condition.
- At V_GS = 4.5V, R_DS(on) might be 0.065 ohm (50% higher!).
- At T_J = 150 degC, R_DS(on) might be 0.090 ohm (double!).

**Always ask:** "Under what conditions was this number measured?"

---
## 5. Typical vs. Min/Max — Know the Difference

| Column | Meaning | Use for design? |
|---|---|---|
| **Min** | Guaranteed worst-case low value | Yes — use for "at least" specs |
| **Typ** | Average over many devices at nominal conditions | No — informational only |
| **Max** | Guaranteed worst-case high value | Yes — use for "at most" specs |

- **Typical is marketing.** It tells you what the average device does under ideal conditions.
- **Min/Max are the guarantee.** These are what the manufacturer tests to and ships against.
- If a parameter only has a "typical" value with no min or max, **the manufacturer is not
  guaranteeing it.** Design with a healthy margin.

---
## 6. Thermal Design Basics

Every semiconductor has a **junction temperature** (T_J) limit — typically 150 degC or
175 degC. Exceeding this destroys the device.

The thermal resistance chain:

```
T_J = T_A + P_D * theta_JA
```

Where:
- **T_J** = junction temperature (degC)
- **T_A** = ambient temperature (degC)
- **P_D** = power dissipated in the device (W)
- **theta_JA** = junction-to-ambient thermal resistance (degC/W)

With a heatsink, the chain splits:

```
T_J = T_A + P_D * (theta_JC + theta_CS + theta_SA)
```

- **theta_JC** = junction-to-case (from datasheet)
- **theta_CS** = case-to-heatsink (depends on interface: thermal paste, pad, etc.)
- **theta_SA** = heatsink-to-ambient (from heatsink datasheet)

In [ ]:
# ============================================================
# Thermal Design Calculator
# ============================================================

def thermal_calculator(P_D, theta_JA, T_A, T_J_max=150.0):
    """
    Calculate junction temperature and check thermal safety.

    Parameters
    ----------
    P_D : float
        Power dissipation in watts.
    theta_JA : float
        Junction-to-ambient thermal resistance in degC/W.
    T_A : float
        Ambient temperature in degC.
    T_J_max : float
        Maximum allowed junction temperature in degC (default 150).

    Returns
    -------
    dict with T_J, margin, max allowable P_D, and safety verdict.
    """
    T_J = T_A + P_D * theta_JA
    margin = T_J_max - T_J
    P_D_max = (T_J_max - T_A) / theta_JA
    safe = T_J < T_J_max

    return {
        "T_J (degC)": round(T_J, 1),
        "T_J_max (degC)": T_J_max,
        "Margin (degC)": round(margin, 1),
        "Max allowable P_D (W)": round(P_D_max, 3),
        "SAFE": safe,
    }


def thermal_calculator_heatsink(P_D, theta_JC, theta_CS, theta_SA, T_A, T_J_max=150.0):
    """
    Calculate junction temperature with a heatsink.

    Parameters
    ----------
    P_D : float
        Power dissipation in watts.
    theta_JC : float
        Junction-to-case thermal resistance (degC/W).
    theta_CS : float
        Case-to-heatsink thermal resistance (degC/W).
    theta_SA : float
        Heatsink-to-ambient thermal resistance (degC/W).
    T_A : float
        Ambient temperature (degC).
    T_J_max : float
        Maximum allowed junction temperature (degC).

    Returns
    -------
    dict with temperatures at each stage and safety verdict.
    """
    theta_total = theta_JC + theta_CS + theta_SA
    T_S = T_A + P_D * theta_SA           # heatsink surface temperature
    T_C = T_S + P_D * theta_CS           # case temperature
    T_J = T_C + P_D * theta_JC           # junction temperature
    margin = T_J_max - T_J
    P_D_max = (T_J_max - T_A) / theta_total

    return {
        "T_A (degC)": T_A,
        "T_heatsink (degC)": round(T_S, 1),
        "T_case (degC)": round(T_C, 1),
        "T_J (degC)": round(T_J, 1),
        "T_J_max (degC)": T_J_max,
        "Margin (degC)": round(margin, 1),
        "theta_total (degC/W)": round(theta_total, 2),
        "Max allowable P_D (W)": round(P_D_max, 3),
        "SAFE": T_J < T_J_max,
    }


# --- Example 1: TO-220 MOSFET with no heatsink ---
print("Example 1: MOSFET in TO-220, no heatsink")
print("  P_D = 2W, theta_JA = 62 degC/W, T_A = 40 degC")
result1 = thermal_calculator(P_D=2.0, theta_JA=62.0, T_A=40.0)
for k, v in result1.items():
    print(f"  {k}: {v}")

print()

# --- Example 2: Same MOSFET with heatsink ---
print("Example 2: Same MOSFET with heatsink")
print("  P_D = 10W, theta_JC = 1.7, theta_CS = 0.5, theta_SA = 8.0, T_A = 40 degC")
result2 = thermal_calculator_heatsink(
    P_D=10.0, theta_JC=1.7, theta_CS=0.5, theta_SA=8.0, T_A=40.0
)
for k, v in result2.items():
    print(f"  {k}: {v}")

In [ ]:
# ============================================================
# Visualize: Junction Temperature vs. Power Dissipation
# for different thermal resistances and ambient temperatures
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Left: T_J vs P_D for different theta_JA ---
ax = axes[0]
P_D_range = np.linspace(0, 5, 100)
T_A = 25.0
T_J_max = 150.0

for theta in [30, 50, 62, 100, 150]:
    T_J = T_A + P_D_range * theta
    ax.plot(P_D_range, T_J, label=f"$\\theta_{{JA}}$ = {theta} degC/W")

ax.axhline(T_J_max, color="red", linestyle="--", linewidth=2, label=f"T_J max = {T_J_max} degC")
ax.set_xlabel("Power Dissipation, P_D (W)")
ax.set_ylabel("Junction Temperature, T_J (degC)")
ax.set_title(f"T_J vs P_D at T_A = {T_A} degC")
ax.legend(fontsize=8)
ax.set_ylim(0, 200)
ax.grid(True, alpha=0.3)

# --- Right: Max allowable P_D vs T_A (derating curve) ---
ax = axes[1]
T_A_range = np.linspace(0, 140, 200)
theta_JA = 62.0  # typical TO-220 without heatsink

P_D_max = np.maximum((T_J_max - T_A_range) / theta_JA, 0)

ax.plot(T_A_range, P_D_max, "b-", linewidth=2)
ax.fill_between(T_A_range, 0, P_D_max, alpha=0.15, color="blue", label="Safe operating region")
ax.set_xlabel("Ambient Temperature, T_A (degC)")
ax.set_ylabel("Max Allowable Power, P_D (W)")
ax.set_title(f"Power Derating Curve ($\\theta_{{JA}}$ = {theta_JA} degC/W)")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 7. Reading Between the Lines

What the datasheet **does not** say can be just as important as what it does.

### Things to watch for:

- **No min/max given** — only a typical value means the manufacturer is not
  committing to that number. Your production units may vary widely.
- **"Not recommended for new designs"** — the part is heading toward obsolescence.
- **Footnotes** — critical caveats often hide in footnotes. Always read them.
- **Application notes referenced** — these often contain vital design information
  that did not fit in the datasheet.
- **Characterization data vs. guaranteed data** — some graphs are based on
  characterization (statistical sample) not 100% production test.
- **Thermal pad / exposed pad** — if the package has an exposed pad, it usually
  **must** be soldered to a copper area for proper thermal and sometimes electrical operation.
- **Moisture sensitivity level (MSL)** — affects storage and soldering requirements.

---
## 8. Common Traps

These are the mistakes that catch beginners (and sometimes experienced engineers):

| Trap | Why it is dangerous | How to avoid |
|---|---|---|
| Confusing absolute max with operating conditions | Absolute max = destruction threshold, not a design target | Never design to operate at absolute max |
| Ignoring test conditions | A parameter at 25 degC may be 2x worse at 85 degC | Always check the conditions column |
| Assuming typical values are guaranteed | Your production batch might be at the min or max | Use min/max for design, typical for estimation |
| Forgetting thermal derating | A device rated for 2W at 25 degC may only handle 0.5W at 100 degC | Always compute T_J at YOUR ambient temp |
| Not checking pin compatibility | Two parts with the same function may have different pinouts | Verify pinout even for "drop-in" replacements |
| Ignoring input protection limits | Exceeding input clamp currents can damage ICs | Check absolute max on ALL pins, not just power |

In [ ]:
# ============================================================
# Interactive-style: Datasheet Parameter Quick Reference
# Select a device type and see the parameters that matter most.
# ============================================================

# This cell provides a function-based lookup. In a Jupyter environment
# with ipywidgets, you could make this interactive with a dropdown.

parameter_guide = {
    "Diode": {
        "V_F (Forward Voltage)": "Voltage drop when conducting. Determines power loss. Typical: 0.3V (Schottky) to 1.1V (standard Si).",
        "I_F (Forward Current)": "Maximum continuous current. Exceeding this causes overheating.",
        "V_R (Reverse Voltage)": "Maximum voltage the diode can block. Also called V_RRM or PIV.",
        "I_R (Reverse Leakage)": "Current that flows when reverse-biased. Increases with temperature.",
        "t_rr (Reverse Recovery Time)": "Time to stop conducting after switching from forward to reverse. Critical in switching applications.",
        "C_j (Junction Capacitance)": "Capacitance across the junction. Affects high-frequency and switching performance.",
    },
    "BJT": {
        "h_FE (DC Current Gain)": "Ratio of I_C to I_B. Varies widely (50-800). Use minimum for design.",
        "V_CE(sat) (Saturation Voltage)": "Voltage across collector-emitter when fully on. Lower = less power loss in switching.",
        "V_BE (Base-Emitter Voltage)": "Voltage needed to forward-bias the base. Typically 0.6-0.7V for Si.",
        "f_T (Transition Frequency)": "Frequency where current gain drops to 1. Defines high-frequency limit.",
        "SOA (Safe Operating Area)": "Graph showing allowed V_CE vs I_C combinations. Must stay inside for reliability.",
    },
    "MOSFET": {
        "V_GS(th) (Gate Threshold)": "Voltage at which the MOSFET begins to turn on. NOT the voltage for full enhancement.",
        "R_DS(on) (On-Resistance)": "Resistance from drain to source when fully on. Determines conduction loss.",
        "Q_g (Total Gate Charge)": "Charge needed to fully turn on the gate. Determines switching speed and driver requirements.",
        "I_D (Drain Current)": "Maximum continuous drain current. Often given at 25 degC case temp.",
        "SOA (Safe Operating Area)": "V_DS vs I_D envelope. Especially important for linear-mode operation.",
    },
    "Op-Amp": {
        "V_OS (Input Offset Voltage)": "DC voltage difference between inputs when output is zero. Lower = more precise.",
        "A_OL (Open-Loop Gain)": "Gain without feedback. Typically 100-120 dB. Higher = more accurate closed-loop.",
        "GBW (Gain-Bandwidth Product)": "Gain x bandwidth = constant. 1 MHz GBW means gain of 10 up to 100 kHz.",
        "CMRR (Common-Mode Rejection)": "Ability to reject signals common to both inputs. Higher dB = better.",
        "Slew Rate (V/us)": "Maximum rate of output voltage change. Limits large-signal bandwidth.",
    },
    "Logic IC": {
        "V_OH / V_OL (Output Levels)": "Guaranteed high and low output voltages. Must be compatible with downstream inputs.",
        "V_IH / V_IL (Input Thresholds)": "Minimum high and maximum low input voltages recognized. Define noise margins.",
        "t_PD (Propagation Delay)": "Time from input change to output change. Limits maximum clock speed.",
        "I_CC (Supply Current)": "Current drawn from power supply. Determines power consumption.",
        "f_MAX (Max Clock Frequency)": "Highest frequency at which the device operates reliably.",
    },
}


def lookup_parameters(device_type):
    """Print the key parameters for a given device type."""
    if device_type not in parameter_guide:
        print(f"Unknown device type: {device_type}")
        print(f"Available types: {', '.join(parameter_guide.keys())}")
        return
    print(f"\n{'=' * 70}")
    print(f"  Key Datasheet Parameters for: {device_type.upper()}")
    print(f"{'=' * 70}")
    for param, description in parameter_guide[device_type].items():
        print(f"\n  {param}")
        print(f"    {description}")
    print()


# Try it: change the device type to explore different parameter sets
lookup_parameters("MOSFET")
lookup_parameters("Op-Amp")

In [ ]:
# ============================================================
# Quick Reference: Unit Prefixes and Common Symbols
# ============================================================

print("UNIT PREFIXES")
print("=" * 55)
prefixes = [
    ("T", "tera",  "10^12",  "1,000,000,000,000"),
    ("G", "giga",  "10^9",   "1,000,000,000"),
    ("M", "mega",  "10^6",   "1,000,000"),
    ("k", "kilo",  "10^3",   "1,000"),
    ("-", "(unit)","10^0",   "1"),
    ("m", "milli", "10^-3",  "0.001"),
    ("u", "micro", "10^-6",  "0.000001"),
    ("n", "nano",  "10^-9",  "0.000000001"),
    ("p", "pico",  "10^-12", "0.000000000001"),
    ("f", "femto", "10^-15", "0.000000000000001"),
]
print(f"{'Symbol':<8} {'Name':<8} {'Factor':<10} {'Value'}")
print("-" * 55)
for sym, name, factor, val in prefixes:
    print(f"{sym:<8} {name:<8} {factor:<10} {val}")

print("\n")
print("COMMON ELECTRICAL SYMBOLS")
print("=" * 55)
symbols = [
    ("V",       "Voltage",         "Volts (V)"),
    ("I",       "Current",         "Amperes (A)"),
    ("R",       "Resistance",      "Ohms (ohm)"),
    ("P",       "Power",           "Watts (W)"),
    ("C",       "Capacitance",     "Farads (F)"),
    ("L",       "Inductance",      "Henrys (H)"),
    ("f",       "Frequency",       "Hertz (Hz)"),
    ("Q",       "Charge",          "Coulombs (C)"),
    ("theta",   "Thermal resistance", "degC/W"),
    ("T_J",     "Junction temp",   "degC"),
    ("T_A",     "Ambient temp",    "degC"),
    ("dB",      "Decibels",        "ratio (log scale)"),
]
print(f"{'Symbol':<10} {'Quantity':<22} {'Unit'}")
print("-" * 55)
for sym, qty, unit in symbols:
    print(f"{sym:<10} {qty:<22} {unit}")

---
## 9. Try the Method: Worked Example

Imagine you have been handed the datasheet for an **IRF540N** N-channel MOSFET.
Let us walk through the 5-step method.

### Step 1: IDENTIFY
- **Type:** N-channel enhancement-mode MOSFET
- **Package:** TO-220AB
- **Pins:** Gate, Drain, Source (tab connected to Drain)
- **Basic function:** Power switch / amplifier, suited for high-current applications

### Step 2: LIMITS (Absolute Maximum Ratings)
| Parameter | Value | Meaning |
|---|---|---|
| V_DS | 100 V | Max drain-source voltage |
| V_GS | +/- 20 V | Max gate-source voltage |
| I_D (continuous) | 33 A (at T_C = 25 degC) | Max continuous drain current |
| P_D | 130 W (at T_C = 25 degC) | Max power dissipation |
| T_J | 175 degC | Max junction temperature |

### Step 3: TYPICAL OPERATION
- Recommended V_GS for full enhancement: 10 V
- R_DS(on) = 0.044 ohm typical at V_GS = 10V, I_D = 33A
- V_GS(th) = 2.0 to 4.0 V (gate threshold)

### Step 4: CURVES
- Output characteristics: I_D vs V_DS at various V_GS levels
- R_DS(on) vs temperature: roughly doubles from 25 degC to 150 degC
- SOA: safe V_DS and I_D combinations for DC and pulsed operation

### Step 5: APPLICATION
- Use V_GS = 10V for reliable full-on operation
- Gate resistor to limit ringing
- Heatsink required for anything above ~2W continuous

In [ ]:
# ============================================================
# Worked Example: Is the IRF540N safe in this circuit?
#
# Circuit: Switching a 12V / 5A resistive load
# V_GS = 10V, T_A = 50 degC, no heatsink
# ============================================================

# Given parameters (from datasheet)
R_DS_on_25C = 0.044    # ohm at 25 degC (typical)
temp_coeff = 1.8       # R_DS(on) multiplier at 150 degC vs 25 degC
theta_JA = 62.0        # degC/W (TO-220 without heatsink)
T_J_max = 175.0        # degC (absolute max)

# Circuit conditions
I_D = 5.0              # amps (load current)
T_A = 50.0             # degC (ambient temperature)

# Step 1: Power dissipation at 25 degC R_DS(on)
P_D_25C = I_D**2 * R_DS_on_25C
print(f"Power dissipation (at 25 degC R_DS(on)): {P_D_25C:.2f} W")

# Step 2: Junction temperature with this power
T_J_initial = T_A + P_D_25C * theta_JA
print(f"Junction temperature (initial estimate):  {T_J_initial:.1f} degC")

# Step 3: At that T_J, R_DS(on) is higher. Iterate.
# Use linear interpolation of temperature coefficient:
# R_DS(on) at T_J approx = R_DS(on)_25C * (1 + (temp_coeff - 1) * (T_J - 25) / (150 - 25))

T_J_est = T_J_initial
for iteration in range(5):
    ratio = 1.0 + (temp_coeff - 1.0) * (T_J_est - 25.0) / (150.0 - 25.0)
    R_DS_hot = R_DS_on_25C * ratio
    P_D_hot = I_D**2 * R_DS_hot
    T_J_est = T_A + P_D_hot * theta_JA

print(f"\nAfter iterating for thermal equilibrium:")
print(f"  R_DS(on) at operating T_J: {R_DS_hot:.4f} ohm")
print(f"  Power dissipation:         {P_D_hot:.2f} W")
print(f"  Junction temperature:      {T_J_est:.1f} degC")
print(f"  T_J max:                   {T_J_max:.1f} degC")
print(f"  Margin:                    {T_J_max - T_J_est:.1f} degC")

if T_J_est < T_J_max:
    print("\n  VERDICT: SAFE (but check if margin is adequate for your application)")
else:
    print("\n  VERDICT: UNSAFE! T_J exceeds maximum. Add a heatsink or reduce current.")

if T_J_est > 125:
    print("  WARNING: T_J > 125 degC. Consider a heatsink for reliability.")

---
## Checkpoint Questions

Test your datasheet reading skills.

### Question 1: Identify the Absolute Maximum Ratings

A datasheet excerpt for a hypothetical NPN BJT shows:

| Parameter | Symbol | Value | Unit |
|---|---|---|---|
| Collector-Emitter Voltage | V_CEO | 45 | V |
| Collector Current (continuous) | I_C | 800 | mA |
| Total Power Dissipation (T_A = 25 degC) | P_D | 625 | mW |
| Junction Temperature | T_J | 150 | degC |

And the electrical characteristics table shows:

| Parameter | Conditions | Min | Typ | Max | Unit |
|---|---|---|---|---|---|
| h_FE | I_C = 150 mA, V_CE = 1V | 100 | 300 | - | - |
| V_CE(sat) | I_C = 500 mA, I_B = 50 mA | - | 0.3 | 0.5 | V |

**Questions:**
1. What is the maximum voltage you can apply across the collector-emitter?
2. The h_FE is listed as 100 min, 300 typ. What value should you use for design?
3. If T_A = 60 degC and theta_JA = 200 degC/W, what is the max allowable P_D?

In [ ]:
# ============================================================
# Checkpoint Answers (run after you have attempted them)
# ============================================================

print("Checkpoint Answers")
print("=" * 60)

print("\nQ1: Maximum V_CE = 45V (absolute maximum rating V_CEO).")
print("    This is the destruction threshold, NOT a design target.")
print("    Design for a comfortable margin below this.")

print("\nQ2: Use h_FE = 100 (the MINIMUM guaranteed value).")
print("    Typical (300) is for estimation only. Your specific")
print("    device might have h_FE = 100 and must still work.")

print("\nQ3: Max allowable P_D at T_A = 60 degC:")
T_A_q3 = 60.0
T_J_max_q3 = 150.0
theta_JA_q3 = 200.0
P_D_max_q3 = (T_J_max_q3 - T_A_q3) / theta_JA_q3
print(f"    P_D_max = (T_J_max - T_A) / theta_JA")
print(f"    P_D_max = ({T_J_max_q3} - {T_A_q3}) / {theta_JA_q3}")
print(f"    P_D_max = {P_D_max_q3:.3f} W = {P_D_max_q3*1000:.0f} mW")
print(f"    (Note: this is LESS than the 625 mW rated at 25 degC!)")

### Question 2: Will This Device Survive?

You have a MOSFET with:
- R_DS(on) = 0.1 ohm (at V_GS = 10V, T_J = 25 degC)
- I_D(max) = 20A
- T_J(max) = 150 degC
- theta_JA = 50 degC/W (no heatsink)

Your circuit runs it at I_D = 8A, V_GS = 10V, T_A = 45 degC.

**Will the device survive?** Calculate T_J.

In [ ]:
# ============================================================
# Checkpoint 2: Calculate and check
# ============================================================

# Student: try to calculate before running this cell!

R_DS_on = 0.1       # ohm
I_D_ck = 8.0        # A
T_A_ck = 45.0       # degC
theta_JA_ck = 50.0  # degC/W
T_J_max_ck = 150.0  # degC

P_D_ck = I_D_ck**2 * R_DS_on
T_J_ck = T_A_ck + P_D_ck * theta_JA_ck

print(f"P_D = I_D^2 * R_DS(on) = {I_D_ck}^2 * {R_DS_on} = {P_D_ck:.1f} W")
print(f"T_J = T_A + P_D * theta_JA = {T_A_ck} + {P_D_ck:.1f} * {theta_JA_ck} = {T_J_ck:.1f} degC")
print(f"T_J_max = {T_J_max_ck} degC")
print()
if T_J_ck >= T_J_max_ck:
    print(f"RESULT: T_J ({T_J_ck:.1f} degC) >= T_J_max ({T_J_max_ck} degC)")
    print("The device will NOT survive! It will overheat and fail.")
    print("Solution: add a heatsink, reduce current, or choose a lower R_DS(on) MOSFET.")
else:
    print(f"RESULT: T_J ({T_J_ck:.1f} degC) < T_J_max ({T_J_max_ck} degC)")
    print(f"The device survives with {T_J_max_ck - T_J_ck:.1f} degC margin.")
    print("But remember: R_DS(on) increases with temperature, so the actual")
    print("T_J will be even higher. Always iterate or add margin.")

---
## 10. Summary: The Datasheet Reading Checklist

Print this out and keep it by your workbench:

1. **IDENTIFY** the device type, package, pinout, and basic function
2. **LIMITS** — note every absolute maximum rating; never design to these values
3. **TYPICAL OPERATION** — find recommended operating conditions and key specs at those conditions
4. **CURVES** — study how parameters change with temperature, voltage, current, frequency
5. **APPLICATION** — use the manufacturer's reference circuit as a starting point

**Golden rules:**
- Use **min/max** for design, **typical** for estimation.
- Always check the **test conditions** for every parameter.
- Always calculate **junction temperature** at your worst-case ambient.
- Always read the **footnotes**.
- When in doubt, add **margin**.

---

*You now have a method that works for any semiconductor datasheet. The next notebook
is the capstone project, where you will put this method into practice on a component
you have never seen before.*